In [3]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException
from datetime import datetime
import pandas as pd
import time
import os

In [4]:
service = Service(executable_path='/Applications/Chdriver/chromedriver')
driver = webdriver.Chrome(service=service)

url = 'https://www.cricbuzz.com/cricket-series/2058/indian-premier-league-2008/matches'
driver.get(url)

match_urls = []
matches = driver.find_elements(By.CLASS_NAME, 'cb-series-matches')
for match in matches:
    link_element = match.find_element(By.CLASS_NAME, 'text-hvr-underline').get_attribute('href')
    link = link_element.replace("cricket-scores", "live-cricket-scorecard")
    match_urls.append(link)

batter_data = {
    'Match Date': [],
    'Venue': [],
    'Link': [],
    'Runs': [],
    'Balls': [],
    'Fours': [],
    'Sixes': [],
    'Strike Rate': [],
    'Team': [],
    'Season': []
}

bowler_data = {
    'Match Date': [],
    'Venue': [],
    'Link': [],
    'Overs': [],
    'Maiden': [],
    'Runs': [],
    'Wicket': [],
    'No Ball': [],
    'Wide': [],
    'Economy': [],
    'Team': [],
    'Season': []
}

for url in match_urls:
    driver.get(url)
    time.sleep(1)

    try:
        venue = driver.find_element(By.XPATH, '//*[@id="page-wrapper"]/div[4]/div[1]/div[2]/a[2]/span/span[2]/span')
        
        season_element = driver.find_element(By.XPATH, '//*[@id="page-wrapper"]/div[4]/div[1]/div[2]/a[1]/span')
        season_words = season_element.text.split()[3:]
        season = ' '.join(season_words)
        
        date = driver.find_element(By.CSS_SELECTOR, 'span.schedule-date')
        date_object = pd.to_datetime(date.text, format='%A, %B %d, %Y')
        
        batting1 = driver.find_element(By.XPATH, '//*[@id="innings_1"]/div[1]')
        batting2 = driver.find_element(By.XPATH, '//*[@id="innings_2"]/div[1]')
        
        xpath1 = '//*[@id="innings_1"]/div[1]/div[1]/span[1]'
        xpath2 = '//*[@id="innings_2"]/div[1]/div[1]/span[1]'
        for section, team_xpath in zip([batting1, batting2], [xpath1, xpath2]):
            team_element = section.find_element(By.XPATH, team_xpath)
            team_words = team_element.text.split(' ')[:-1]
            initials = [word[0] for word in team_words]
            team_name = ''.join(initials)
            
            batters = section.find_elements(By.CLASS_NAME, 'cb-scrd-itms')
            for batter in batters[:-3]:
                batter_data['Venue'].append(venue.text)                
                batter_data['Season'].append(season)
                batter_data['Match Date'].append(date_object.strftime('%Y-%m-%d'))

                name_link = batter.find_element(By.CLASS_NAME, 'cb-text-link').get_attribute('href')
                batter_data['Link'].append(name_link)
                                
                scores = batter.find_elements(By.CLASS_NAME, 'text-right')
                batter_data['Runs'].append(scores[0].text)
                batter_data['Balls'].append(scores[1].text)
                batter_data['Fours'].append(scores[2].text)
                batter_data['Sixes'].append(scores[3].text)
                batter_data['Strike Rate'].append(scores[4].text)
                batter_data['Team'].append(team_name)
            
        bowling1 = driver.find_element(By.XPATH, '//*[@id="innings_1"]/div[4]')
        bowling2 = driver.find_element(By.XPATH, '//*[@id="innings_2"]/div[4]')
        xpath1 = '//*[@id="innings_1"]/div[1]/div[1]/span[1]'
        xpath2 = '//*[@id="innings_2"]/div[1]/div[1]/span[1]'
        for section, team_xpath in zip([bowling1, bowling2], [xpath2, xpath1]):
            team_element = section.find_element(By.XPATH, team_xpath)
            team_words = team_element.text.split(' ')[:-1]
            initials = [word[0] for word in team_words]
            team_name = ''.join(initials)
            
            bowlers = section.find_elements(By.CLASS_NAME, 'cb-scrd-itms')
            for bowler in bowlers:
                name_link = bowler.find_element(By.CLASS_NAME, 'cb-text-link').get_attribute('href')   
                bowler_data['Link'].append(name_link)

                bowler_data['Match Date'].append(date_object.strftime('%Y-%m-%d'))               
                bowler_data['Venue'].append(venue.text)
                bowler_data['Season'].append(season)

                scores = bowler.find_elements(By.CLASS_NAME, 'text-right')
                bowler_data['Overs'].append(scores[0].text)
                bowler_data['Maiden'].append(scores[1].text)
                bowler_data['Runs'].append(scores[2].text)
                bowler_data['Wicket'].append(scores[3].text)
                bowler_data['No Ball'].append(scores[4].text)
                bowler_data['Wide'].append(scores[5].text)
                bowler_data['Economy'].append(scores[6].text)
                bowler_data['Team'].append(team_name)
                
    except NoSuchElementException as e:
        pass

batter_df = pd.DataFrame(batter_data)
bowler_df = pd.DataFrame(bowler_data)


driver.quit()

In [5]:
batter_link = pd.DataFrame({'Link': batter_df['Link'].unique()})
bowler_link = pd.DataFrame({'Link': bowler_df['Link'].unique()})
player_link = pd.concat([batter_link, bowler_link], axis=0).reset_index(drop=True)['Link'].unique()

In [6]:
service = Service(executable_path='/Applications/Chdriver/chromedriver')
driver = webdriver.Chrome(service=service)

player_data = {
    'Link': [],
    'Name': [],
    'Country': [],
    'Born': []
}
for link in player_link:
    driver.get(link)
    time.sleep(0.1)
    
    player_data['Link'].append(link)
    
    name = driver.find_element(By.XPATH, '//*[@id="playerProfile"]/div[1]/div[2]/h1').text
    player_data['Name'].append(name)

    country = driver.find_element(By.XPATH, '//*[@id="playerProfile"]/div[1]/div[2]/h3').text
    player_data['Country'].append(country)

    date_string = driver.find_element(By.XPATH, '//*[@id="playerProfile"]/div[2]/div[1]/div/div[3]').text
    date_part = date_string.split('(')[0].strip()
    parsed_date = datetime.strptime(date_part, '%b %d, %Y')
    formatted_date = parsed_date.strftime('%Y-%m-%d')
    player_data['Born'].append(formatted_date)
    
player_df = pd.DataFrame(player_data)    
driver.quit()

In [7]:
player_df

,Link,Name,Country,Born
0,https://www.cricbuzz.com/profiles/29/sourav-ga...,Sourav Ganguly,India,1972-07-08
1,https://www.cricbuzz.com/profiles/201/brendon-...,Brendon McCullum,New Zealand,1981-09-27
2,https://www.cricbuzz.com/profiles/38/ricky-pon...,Ricky Ponting,Australia,1974-12-19
3,https://www.cricbuzz.com/profiles/562/david-hu...,David Hussey,Australia,1977-07-15
4,https://www.cricbuzz.com/profiles/360/mohammad...,Mohammad Hafeez,Pakistan,1980-10-17
...,...,...,...,...
154,https://www.cricbuzz.com/profiles/205/daniel-v...,Daniel Vettori,New Zealand,1979-01-27
155,https://www.cricbuzz.com/profiles/1435/brett-g...,Brett Geeves,Australia,1982-06-13
156,https://www.cricbuzz.com/profiles/1822/gagande...,Gagandeep Singh,India,1981-10-03
157,https://www.cricbuzz.com/profiles/30/shoaib-ak...,Shoaib Akhtar,Pakistan,1975-08-13


In [8]:
df1 = pd.merge(batter_df, player_df, on='Link', how='left')
df2 = pd.merge(bowler_df, player_df, on='Link', how='left')
df1 = df1[['Match Date', 'Name', 'Runs', 'Balls','Fours', 'Sixes', 'Strike Rate', 'Country', 'Born', 'Team', 'Venue', 'Season']]
df2 = df2[['Match Date', 'Name', 'Overs', 'Maiden','Runs', 'Wicket', 'No Ball', 'Wide', 'Economy', 'Country', 'Born', 'Team', 'Venue', 'Season']]

In [9]:
batter_file = 'ipl-2008-batter.csv'
bowler_file = 'ipl-2008-bowler.csv'

df1.to_csv(batter_file, index=False, header=not os.path.isfile(batter_file))
df2.to_csv(bowler_file, index=False, header=not os.path.isfile(bowler_file))


In [10]:
df = pd.read_csv(batter_file)
df

,Match Date,Name,Runs,Balls,Fours,Sixes,Strike Rate,Country,Born,Team,Venue,Season
0,2008-04-18,Sourav Ganguly,10,12,2,0,83.33,India,1972-07-08,KKR,Bengaluru,2008
1,2008-04-18,Brendon McCullum,158,73,10,13,216.44,New Zealand,1981-09-27,KKR,Bengaluru,2008
2,2008-04-18,Ricky Ponting,20,20,1,1,100.00,Australia,1974-12-19,KKR,Bengaluru,2008
3,2008-04-18,David Hussey,12,12,1,0,100.00,Australia,1977-07-15,KKR,Bengaluru,2008
4,2008-04-18,Mohammad Hafeez,5,3,1,0,166.67,Pakistan,1980-10-17,KKR,Bengaluru,2008
...,...,...,...,...,...,...,...,...,...,...,...,...
802,2008-06-01,Yusuf Pathan,56,39,3,4,143.59,India,1982-11-17,RR,Mumbai,2008
803,2008-06-01,Mohammad Kaif,12,9,0,1,133.33,India,1980-12-01,RR,Mumbai,2008
804,2008-06-01,Ravindra Jadeja,0,1,0,0,0.00,India,1988-12-06,RR,Mumbai,2008
805,2008-06-01,Shane Warne,9,9,1,0,100.00,Australia,1969-09-13,RR,Mumbai,2008
